In [3]:
from concurrent.futures import ProcessPoolExecutor
from datetime import date, datetime, timedelta
import time

import geopandas as gpd
import pandas as pd
import os
import numpy as np
from tabulate import tabulate
from prettytable import PrettyTable
import fiona

import matplotlib.pyplot as plt
import seaborn as sns

import rtree

from PIL import Image

import brewer2mpl

import requests
import zipfile

from shapely.geometry import Point, Polygon, MultiPolygon, MultiLineString
from shapely.ops import unary_union
from shapely.wkt import loads
import rasterio as rio
import rasterio.mask
from rasterio.plot import show
from shapely.geometry import mapping
from rasterio.transform import Affine


In [26]:
western_mass_2021_dissolve = gpd.read_file('../results/csv/01_19_2024/dissolve_split/western_mass_2021_dissolve_split.csv', GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")

western_mass_2019_dissolve = gpd.read_file('../results/csv/01_19_2024/dissolve_split/western_mass_2019_dissolve_split.csv', GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")

eastern_mass_2021_dissolve = gpd.read_file('../results/csv/01_19_2024/dissolve_split/eastern_mass_2021_dissolve_split.csv', GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")

eastern_mass_2019_dissolve = gpd.read_file('../results/csv/01_19_2024/dissolve_split/eastern_mass_2019_dissolve_split.csv', GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO")

ValueError: GeoDataFrame does not support multiple columns using the geometry column name 'geometry'.

In [65]:
def export_complete_product(gdf, dissolve=False):
    # List of columns to drop
    columns_to_drop = ['intersection_points', 'intersecting_roads', 'non_intersecting_segments', 'road_geometry_buffer']
    # Drop specified columns
    gdf_filtered = gdf.drop(columns=columns_to_drop)
    gdf_filtered = gdf_filtered[~((gdf_filtered['type'] == 'other') | (gdf_filtered['type'] == 'parking') | (gdf_filtered['category'] == 'fp'))]
    file_name = f"{gdf_filtered['source'].iloc[0]}_{gdf_filtered['year'].iloc[0]}"
    print(file_name)
    if(dissolve):
        road_type = 'dissolve'
    else:
        road_type = 'road_arc'
    
    # Specify the directory path
    directory_path = f'../{road_type}/'

    # Create the directory if it doesn't exist
    os.makedirs(directory_path, exist_ok=True)
    
    complete_file_name = f"{file_name}_{road_type}.gdb"
    # Set CRS to NAD 1983 MA State Plane system
    print(gdf_filtered.crs)
    if str.split(file_name,'_')[0] == 'western':
        epsg_code = 'EPSG:6347'
    else:
        epsg_code = 'EPSG:6348'
    #epsg_code = 'EPSG:4269' # NAD83
    gdf_filtered = gdf_filtered.set_crs(epsg_code)    
    gdf_filtered = gdf_filtered.to_crs(epsg_code)
    gdf_filtered.to_file(os.path.join(directory_path, complete_file_name),layer=file_name+'_'+road_type, driver='OpenFileGDB')

In [66]:
dissolved_results = [western_mass_2019_dissolve, eastern_mass_2019_dissolve, western_mass_2021_dissolve, eastern_mass_2021_dissolve]
for dr in dissolved_results:
    export_complete_product(dr, True)

western_mass_2019
None
eastern_mass_2019
None
western_mass_2021
None
eastern_mass_2021
None


In [60]:
def filter_gdfs_project_4269(gdf, dissolve=False):
    # List of columns to drop
    columns_to_drop = ['intersection_points', 'intersecting_roads', 'non_intersecting_segments', 'road_geometry_buffer']
    # Drop specified columns
    gdf_filtered = gdf.drop(columns=columns_to_drop)
    gdf_filtered = gdf_filtered[~((gdf_filtered['type'] == 'other') | (gdf_filtered['type'] == 'parking') | (gdf_filtered['category'] == 'fp'))]
    print(gdf_filtered.crs)
    file_name = f"{gdf_filtered['source'].iloc[0]}_{gdf_filtered['year'].iloc[0]}"
    print(file_name)
    if str.split(file_name,'_')[0] == 'western':
        epsg_code = 'EPSG:6347' # Western MA state-plane coordinate system
    else:
        epsg_code = 'EPSG:6348'  # Eastern MA state-plane coordinate system   
    gdf_filtered = gdf_filtered.set_crs(epsg_code)    
    gdf_filtered = gdf_filtered.to_crs(epsg_code)   
    epsg_code = 'EPSG:4269' # NAD83    
    #gdf_filtered = gdf_filtered.set_crs(epsg_code)    
    gdf_filtered = gdf_filtered.to_crs(epsg_code)
    print(gdf_filtered.crs)
    return gdf_filtered

In [61]:
wm19_4269 = filter_gdfs_project_4269(western_mass_2019_dissolve, True)
em19_4269 = filter_gdfs_project_4269(eastern_mass_2019_dissolve, True)
wm21_4269 = filter_gdfs_project_4269(western_mass_2021_dissolve, True)
em21_4269 = filter_gdfs_project_4269(eastern_mass_2021_dissolve, True)

None
western_mass_2019
EPSG:4269
None
eastern_mass_2019
EPSG:4269
None
western_mass_2021
EPSG:4269
None
eastern_mass_2021
EPSG:4269


In [62]:
def export_combined_gdf(gdflist, dissolve=False):
    combined_results = pd.concat(gdflist, ignore_index=True)    
    file_name = f"combined_mass_{combined_results['year'].unique()[0]}"
    # Specify the directory path
    if(dissolve):
        road_type = 'dissolve'
    else:
        road_type = 'road_arc'    
    directory_path = f'../results/layers/{road_type}/'

    # Create the directory if it doesn't exist
    os.makedirs(directory_path, exist_ok=True)
    
    complete_file_name = f"{file_name}_{road_type}.gdb"
    # Set CRS to NAD 1983 MA State Plane system
    print(combined_results.crs)

    combined_results.to_file(os.path.join(directory_path, complete_file_name),layer=file_name+'_'+road_type, driver='OpenFileGDB')

In [63]:
export_combined_gdf([wm19_4269,em19_4269], True)

EPSG:4269


In [64]:
export_combined_gdf([wm21_4269,em21_4269], True)

EPSG:4269
